Long term goal: use this notebook at training data to mangle/uglify/transpile any language


demonstrate patterns between different languages, in C# interfaces are like:

```
interface MyInterface {
    myProperty: string
}

class MyClass {
}
```

in javascript the same effect can be accomplished with a polyfill

```
function applyInterface(class) {
    var properties = ['myProperty'];
    var i, result = {};
    for(i=0; i<properties.length; i++) {
        if(typeof class[properties[i]] === 'undefined') {
            throw new Error(`Missing interface property ${properties[i]}`);
        }
        result[properties[i]] = class[properties[i]];
    }
}
```

